<a href="https://colab.research.google.com/github/eugene123tw/cs330-hw1/blob/master/HW1_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Downlaod Omniglot 
workspace_dir = '.'
!gdown --id 1aBacYkuigdlKExME-kgxqworbdd8Zixg --output "{workspace_dir}/omniglot_resized.zip"
!unzip -q omniglot_resized

Downloading...
From: https://drive.google.com/uc?id=1aBacYkuigdlKExME-kgxqworbdd8Zixg
To: /content/omniglot_resized.zip
13.0MB [00:00, 49.0MB/s]


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Apr 11 14:30:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import os
import random

import numpy as np
from skimage.io import imread

import torch
from torch import nn

cuda = torch.device('cuda')

In [0]:
def get_images(paths, labels, nb_samples=None, shuffle=True):
  """
  Takes a set of character folders and labels and returns paths to image files
  paired with labels.
  Args:
      paths: A list of character folders
      labels: List or numpy array of same length as paths
      nb_samples: Number of images to retrieve per character
  Returns:
      List of (label, image_path) tuples
  """
  if nb_samples is not None:
    sampler = lambda x: random.sample(x, nb_samples)
  else:
    sampler = lambda x: x
  images_labels = [(i, os.path.join(path, image))
                   for i, path in zip(labels, paths)
                   for image in sampler(os.listdir(path))]
  if shuffle:
    random.shuffle(images_labels)
  return images_labels


def image_file_to_array(filename, dim_input):
  """
  Takes an image path and returns numpy array
  Args:
      filename: Image filename
      dim_input: Flattened shape of image
  Returns:
      1 channel image
  """
  image = imread(filename)
  image = image.reshape([dim_input])
  image = image.astype(np.float32) / 255.0
  image = 1.0 - image
  return image


class DataGenerator(object):
  """
  Data Generator capable of generating batches of Omniglot data.
  A "class" is considered a class of omniglot digits.
  """

  def __init__(self, num_classes, num_samples_per_class, config={}):
    """
    Args:
        num_classes: Number of classes for classification (K-way)
        num_samples_per_class: num samples to generate per class in one batch
        batch_size: size of meta batch size (e.g. number of functions)
    """
    self.num_samples_per_class = num_samples_per_class
    self.num_classes = num_classes

    data_folder = config.get('data_folder', './omniglot_resized')
    self.img_size = config.get('img_size', (28, 28))

    self.dim_input = np.prod(self.img_size)
    self.dim_output = self.num_classes

    character_folders = [os.path.join(data_folder, family, character)
                         for family in os.listdir(data_folder)
                         if os.path.isdir(os.path.join(data_folder, family))
                         for character in os.listdir(os.path.join(data_folder, family))
                         if os.path.isdir(os.path.join(data_folder, family, character))]

    random.seed(1)
    random.shuffle(character_folders)
    num_val = 100
    num_train = 1100
    self.metatrain_character_folders = character_folders[: num_train]
    self.metaval_character_folders = character_folders[
                                     num_train:num_train + num_val]
    self.metatest_character_folders = character_folders[
                                      num_train + num_val:]

  def sample_batch(self, batch_type, batch_size):
    """
    Samples a batch for training, validation, or testing
    Args:
        batch_type: train/val/test
    Returns:
        A a tuple of (1) Image batch and (2) Label batch where
        image batch has shape [B, K, N, 784] and label batch has shape [B, K, N, N]
        where B is batch size, K is number of samples per class, N is number of classes
    """
    if batch_type == "train":
      folders = self.metatrain_character_folders
    elif batch_type == "val":
      folders = self.metaval_character_folders
    else:
      folders = self.metatest_character_folders

    #############################
    #### YOUR CODE GOES HERE ####
    # Initialise array for storage
    all_image_batches = np.zeros(
      (batch_size, self.num_samples_per_class, self.num_classes, 784)
    )

    all_label_batches = np.zeros(
      (batch_size, self.num_samples_per_class, self.num_classes, self.num_classes)
    )

    for b in range(batch_size):  # sample mini batch of tasks
      sampled_paths = np.random.choice(folders, self.num_classes)
      images_labels = get_images(
        sampled_paths, np.eye(self.num_classes), self.num_samples_per_class, False)

      test_images, test_labels = [], []
      train_images, train_labels = [], []
      for i, (y_vector, image_path) in enumerate(images_labels):
        image = image_file_to_array(image_path, 784)
        if i % self.num_samples_per_class == 0:
          test_images.append(image)
          test_labels.append(y_vector)
        else:
          train_images.append(image)
          train_labels.append(y_vector)

      ts_indices = np.random.permutation(range(len(test_images)))
      test_images = np.array(test_images)[ts_indices]
      test_labels = np.array(test_labels)[ts_indices]

      tr_indices = np.random.permutation(range(len(train_images)))
      train_images = np.array(train_images)[tr_indices]
      train_labels = np.array(train_labels)[tr_indices]

      all_image_batches[b, ...] = np.vstack((train_images, test_images)).reshape(
        self.num_samples_per_class, self.num_classes, -1
      )
      all_label_batches[b, ...] = np.vstack((train_labels, test_labels)).reshape(
        self.num_samples_per_class, self.num_classes, -1)
    #############################

    return all_image_batches, all_label_batches

In [0]:
class MANN(nn.Module):
  def __init__(self, num_classes):
    super(MANN, self).__init__()
    # The first axis is the sequence itself, 
    # the second indexes instances in the mini-batch, 
    # and the third indexes elements of the input
    self.N = num_classes
    self.layer1 = nn.LSTM(784 + num_classes, 128)
    self.layer2 = nn.LSTM(128, num_classes)
  
  def forward(self, x):
    """ x is a tensor concatenated images and labels: [(K + 1)*N, B, 784 + N] """
    # Hint: Passing zeros, not the ground truth labels for the final N examples.
    #       Remember we should mask the true labels of test set. 
    #       Otherwise, our model will learn the pattern and we don't want this.
    x[-self.N:, :, -self.N:] = 0
    out, _ = self.layer1(x)
    out, _ = self.layer2(out)
    return out

# Start training

In [0]:
import argparse

parser = argparse.ArgumentParser(description='Process some flags.')

parser.add_argument('--num_classes', type=int, default=2, help='number of classes used in classification (e.g. 5-way classification).')

parser.add_argument('--num_samples', type=int, default=1, help='number of examples used for inner gradient update (K for K-shot learning).')

parser.add_argument('--meta_batch_size', type=int, default=4, help='Number of N-way classification tasks per batch')

parser.add_argument('--data_root', type=str, default= 'omniglot_resized', help='data folder root')

FLAGS = parser.parse_args(args=[])

In [0]:
import torch.optim as optim
net = MANN(num_classes=FLAGS.num_classes)
net = net.cuda().train()
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [12]:
data_generator = DataGenerator(
  FLAGS.num_classes,
  FLAGS.num_samples + 1,
  {'data_folder': FLAGS.data_root}
)

for step in range(100000):
  net = net.cuda().train()
  images, labels = data_generator.sample_batch('train', FLAGS.meta_batch_size)
  B, K_plus_1, N, _ = images.shape

  images = images.reshape((B, K_plus_1 * N, -1))
  labels = labels.reshape((B, K_plus_1 * N, N))

  images = torch.tensor(images, device=cuda)
  labels = torch.tensor(labels, device=cuda)

  optimizer.zero_grad()   # zero the gradient buffers
  input_tensor = torch.tensor(torch.cat([images, labels], dim=2).transpose(0, 1), device=cuda)
  logits = net(input_tensor.float()).transpose(0, 1)  
  # only compute loss for test data which are the final N points
  # logits: [B, (K + 1) * N , N]
  # labels: [B, (K + 1) * N , N]
  loss = criterion(logits[:, -N:], labels[:, -N:].argmax(axis=1))
  loss.backward()
  optimizer.step()    

  if step % 1000 == 0:
    with torch.no_grad():
      print("*" * 5 + "Iter " + str(step) + "*" * 5)
      net.eval()
      images, labels = data_generator.sample_batch('test', 100)
      B, K_plus_1, N, _ = images.shape

      images = images.reshape((B, K_plus_1 * N, -1))
      labels = labels.reshape((B, K_plus_1 * N, N))

      images = torch.tensor(images, device=cuda)
      labels = torch.tensor(labels, device=cuda)
      input_tensor = torch.tensor(torch.cat([images, labels], dim=2).transpose(0, 1), device=cuda)
      
      logits = net(input_tensor.float()).transpose(0, 1)
      test_loss = criterion(logits[:, -N:], labels[:, -N:].argmax(axis=1))
      test_loss_value = test_loss.detach().cpu().numpy()

      loss_value = loss.detach().cpu().numpy()

      print("Train Loss:", loss_value, "Test Loss:", test_loss_value)
      logits = logits.reshape(
        -1, FLAGS.num_samples + 1,
        FLAGS.num_classes, FLAGS.num_classes)
      
      labels = labels.reshape((B, K_plus_1, N, N))

      logits = logits[:, -1, :, :].argmax(2)
      labels = labels[:, -1, :, :].argmax(2)
      accuracy = (1.0 * (logits == labels)).mean()
      print("Test Accuracy", accuracy.detach().cpu().numpy())

save_path = "/content/drive/My Drive/cs330/hw1/{}.pth".format("MANN")
torch.save(net.state_dict(), save_path)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


*****Iter 0*****
Train Loss: 0.6931702 Test Loss: 0.6931433
Test Accuracy 0.49499997
*****Iter 1000*****
Train Loss: 0.693148 Test Loss: 0.69314224
Test Accuracy 0.5
*****Iter 2000*****
Train Loss: 0.69314307 Test Loss: 0.6931455
Test Accuracy 0.505
*****Iter 3000*****
Train Loss: 0.6930448 Test Loss: 0.6931305
Test Accuracy 0.5
*****Iter 4000*****
Train Loss: 0.69335043 Test Loss: 0.69312114
Test Accuracy 0.5
*****Iter 5000*****
Train Loss: 0.6930114 Test Loss: 0.69327897
Test Accuracy 0.5
*****Iter 6000*****
Train Loss: 0.6932169 Test Loss: 0.6931318
Test Accuracy 0.5
*****Iter 7000*****
Train Loss: 0.69316673 Test Loss: 0.6931458
Test Accuracy 0.5
*****Iter 8000*****
Train Loss: 0.6931713 Test Loss: 0.693145
Test Accuracy 0.5
*****Iter 9000*****
Train Loss: 0.69314456 Test Loss: 0.6931498
Test Accuracy 0.5
*****Iter 10000*****
Train Loss: 0.6931485 Test Loss: 0.6931461
Test Accuracy 0.5
*****Iter 11000*****
Train Loss: 0.6931375 Test Loss: 0.69314694
Test Accuracy 0.5
*****Iter 1200